In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Flatten, Dense
from tensorflow.keras.models import Model

In [2]:
DATASET = datasets.cifar10.load_data()
(X_train, y_train), (X_test, y_test) = DATASET
# X_train, X_valid = train_test_split(X_train, test_size=0.2, random_state=42)
# y_train, y_valid = train_test_split(y_train, test_size=0.2, random_state=42)

In [3]:
image_size = 32
epochs = 1
batch_size = 8

In [4]:
(X_train[0].shape, y_train.shape)

((32, 32, 3), (50000, 1))

In [5]:
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x


def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    x9 = Flatten() (x8)
    d9 = Dense(num_classes, activation="softmax") (x9)
    # output = Conv2D(num_classes, 1, padding="same", activation="softmax")(d9)

    return Model(inputs, d9)

In [6]:
shape = (32, 32, 3)
num_classes = 10
model = build_unet(shape, num_classes)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [8]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 218s 138ms/step - loss: 1.3945 - acc: 0.5039
Epoch 2/5
1563/1563 [==============================] - 214s 137ms/step - loss: 0.9868 - acc: 0.6539
Epoch 3/5
1563/1563 [==============================] - 211s 135ms/step - loss: 0.8121 - acc: 0.7168
Epoch 4/5
1563/1563 [==============================] - 215s 138ms/step - loss: 0.6901 - acc: 0.7609
Epoch 5/5
1563/1563 [==============================] - 216s 138ms/step - loss: 0.5918 - acc: 0.7931


KeyboardInterrupt: 

In [9]:
test_loss, test_acc = model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 30ms/step - loss: 0.8119 - acc: 0.7208
